This notebook replicates the experimental setup used in the comparison presented in the corresponding Esann paper found here:
https://pub.uni-bielefeld.de/publication/2908201

requires borutyPy:
https://github.com/scikit-learn-contrib/boruta_py

## Define models

In [11]:
import abc
from sklearn.svm import LinearSVC
from sklearn.linear_model import  RidgeClassifier, ElasticNet, SGDClassifier

from boruta import BorutaPy

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectFromModel, mutual_info_classif, GenericUnivariateSelect
from rbclassifier import RelevanceBoundsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import warnings

class FSmodel(object):
    """
    Abstract class for all models which are used for feature selection
    """
    __metaclass__ = abc.ABCMeta
    def __init__(self,X,Y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
        scaler = StandardScaler().fit(self.X_train)
        self.X_train = scaler.transform(self.X_train)
        self.X_test = scaler.transform(self.X_test)
    @abc.abstractmethod
    def predict(self):
        return


class SVCL1(FSmodel):
    def predict(self):
        clf = LinearSVC(penalty="l1",dual=False).fit(self.X_train, self.y_train)
        sfm = SelectFromModel( clf, prefit=True)
        self.score = clf.score(self.X_test,self.y_test)
        self.dec = clf.coef_[0]
        return   sfm.get_support()
class Ridge(FSmodel):
    def predict(self):
        clf = RidgeClassifier().fit(self.X_train, self.y_train)
        sfm = SelectFromModel(clf, prefit=True)
        self.score = clf.score(self.X_test,self.y_test)
        self.dec = clf.coef_[0]
        return sfm.get_support()
class ElasticN(FSmodel):
    def predict(self):
        tuned_parameters = [{'alpha': 0.00001 * np.logspace(0, 3)}]
        clf = GridSearchCV(SGDClassifier(l1_ratio=0.15,penalty="elasticnet"),
                           tuned_parameters,
                            cv=5)
        clf.fit(self.X_train, self.y_train)
        clf = clf.best_estimator_
        sfm = SelectFromModel(clf, prefit=True)
        self.score = clf.score(self.X_test,self.y_test)
        self.dec = clf.coef_[0]
        return sfm.get_support()
class BorutaModel(FSmodel):
    def predict(self):
        rf = RandomForestClassifier(max_depth=5)
        borutaf = BorutaPy(rf, n_estimators="auto", verbose=False)
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', 'invalid value encountered in greater')
            borutaf.fit(self.X_train, self.y_train)
        rf = RandomForestClassifier(max_depth=3).fit(self.X_train, self.y_train)
        self.score = rf.score(self.X_test,self.y_test)
        self.dec = np.zeros(len(self.X_train.T))
        return borutaf.support_

class OurMethod(FSmodel):
    def predict(self):
        self.smo = RelevanceBoundsClassifier()
        self.smo.fit(self.X_train, self.y_train)
        self.score = self.smo._svm_clf.score(self.X_test,self.y_test)
        self.dec =np.zeros(len(self.X_train.T))
        return self.smo.allrel_prediction_



## Data generation method

In [12]:
from rbclassifier.genData import genData
import numpy as np

def data(n=150, d=12, redundant=4, informative=3, repeated = 0, state=1338):
    randomstate = np.random.RandomState(state)
    # Use method provided my library
    X1, Y1 = genData(n_samples=n, n_features=d,
                           n_redundant=redundant,
                           strRel=informative, n_repeated=repeated,
                           class_sep=0.3,
                           flip_y=0)
    # Get a truth vector for accuracy measurement
    truth = [True]*(informative+redundant) + [False]*(d-(informative+redundant))
    strTruth = [True]*(informative) + [False]*(d-(informative))
    weakTruth  = [False]*(informative) + [True]*(redundant) + [False]*(d-(informative+redundant))
    return X1,Y1,truth, strTruth,weakTruth

### Test Setup

In [13]:
# Define which methods to compare
tests = [Ridge,SVCL1,ElasticN,OurMethod,BorutaModel] # Use model names from above

In [14]:
from sklearn.metrics import precision_score,roc_auc_score, classification_report, recall_score, f1_score
import pandas as pd
from IPython.display import display, HTML
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import itertools 


test_names =  [x.__name__ for x in tests]
test_metrics  = ["accuracy","precision","recall","F1 measure","auc"]
test_n = 10
test_range = list(range(1337,1337+test_n))



def singleTest(X1,Y1,truth):
        out =  []
        for x in tests:
            model = x(X1,Y1)
            pred = model.predict()      
            prec = precision_score(truth,pred)
            recall = recall_score(truth,pred)
            F1 = f1_score(truth,pred)
            accuracy = model.score
            auc = roc_auc_score(truth, model.dec)
            out.append([accuracy,prec,recall,F1,auc])
        return out
    


## Run feature selection with various models
test is repeated multple times

### Test for all relevant feature selection performance

In [15]:
# Define a pandas Panel to hold results for later
allRelData = pd.Panel(items=test_names, major_axis=test_range, minor_axis=test_metrics)

# Generate data for experiment
all_rel_test_data = []
for state in test_range:
    all_rel_test_data.append(data(n=150, d=12, redundant=4, informative=3, repeated = 0, state=state))
    
    
# Run single tests in parallel using pool    
with Pool() as p:
    data_iter = [all_rel_test_data[xi][0:3] for xi in range(test_n)]
    parall_result = p.starmap(singleTest,data_iter)
        
# Save results in pandas.Panel        
for i,state in zip(test_range,parall_result):
    for j,x in zip(range(len(tests)),tests):
        allRelData.loc[x.__name__].loc[i] = state[j]

### Test for strongly relevant feature selection performance

In [16]:
strRelData = pd.Panel(items=test_names, major_axis=test_range, minor_axis=test_metrics)
str_rel_test_data = []
for state in test_range:
    str_rel_test_data.append( data(n=150, d=12, redundant=0, informative=6, repeated = 0, state=state))
    
with Pool() as p:
    data_iter = [str_rel_test_data[xi][0:3] for xi in range(test_n)]
    parall_result = p.starmap(singleTest,data_iter)
        
# Save results in pandas.Panel        
for i,state in zip(test_range,parall_result):
    for j,x in zip(range(len(tests)),tests):
        strRelData.loc[x.__name__].loc[i] = state[j]

### Test for weakly relevant feature selection performance

In [17]:
weakRelData = pd.Panel(items=test_names, major_axis=test_range, minor_axis=test_metrics)

weak_rel_test_data = []
for state in test_range:
    weak_rel_test_data.append(  data(n=150, d=12, redundant=6, informative=0, repeated = 0, state=state))
    
with Pool() as p:
    data_iter = [weak_rel_test_data[xi][0:3] for xi in range(test_n)]
    parall_result = p.starmap(singleTest,data_iter)
        
# Save results in pandas.Panel        
for i,state in zip(test_range,parall_result):
    for j,x in zip(range(len(tests)),tests):
        weakRelData.loc[x.__name__].loc[i] = state[j]

## Mutual Information from matlab ( ignore the block if matlab code is missing)


using external matlab implementation for mutual information from Benoît Frénay (https://bfrenay.wordpress.com/mutual-information/)

Save datasets for matlab

In [11]:
import scipy.io as sio

for state,datum in zip(test_range,str_rel_test_data):
    X1, Y1, *_ = datum
    obj_arr = {"x":X1,"y":Y1}
    sio.savemat("../benoit/datasets/s{}.mat".format(state),obj_arr)
for state,datum in zip(test_range,weak_rel_test_data):
    X1, Y1, *_ = datum
    obj_arr = {"x":X1,"y":Y1}
    sio.savemat("../benoit/datasets/w{}.mat".format(state),obj_arr)
for state,datum in zip(test_range,all_rel_test_data):
    X1, Y1, *_ = datum
    obj_arr = {"x":X1,"y":Y1}
    sio.savemat("../benoit/datasets/a{}.mat".format(state),obj_arr)

In [12]:
# RUN MATLAB.....

import results back to python

In [13]:
weak_mat_results = pd.Panel(items=test_range, minor_axis=range(12), major_axis=['rel_lower_MI_bounds', 'rel_upper_MI_bounds','rel_lower_MSE_bounds','rel_upper_MSE_bounds'])
strong_mat_results = pd.Panel(items=test_range, minor_axis=range(12), major_axis=['rel_lower_MI_bounds', 'rel_upper_MI_bounds','rel_lower_MSE_bounds','rel_upper_MSE_bounds'])
all_mat_results = pd.Panel(items=test_range, minor_axis=range(12), major_axis=['rel_lower_MI_bounds', 'rel_upper_MI_bounds','rel_lower_MSE_bounds','rel_upper_MSE_bounds'])


for state in test_range:
    mat = sio.loadmat("../benoit/ares{}.mat".format(state))
    all_mat_results.loc[state] = mat['rel_lower_MI_bounds'],  mat['rel_upper_MI_bounds'],  mat['rel_lower_MSE_bounds'],  mat['rel_upper_MSE_bounds']
    
    mat = sio.loadmat("../benoit/sres{}.mat".format(state))
    strong_mat_results.loc[state] = mat['rel_lower_MI_bounds'],  mat['rel_upper_MI_bounds'],  mat['rel_lower_MSE_bounds'],  mat['rel_upper_MSE_bounds']
    
    mat = sio.loadmat("../benoit/wres{}.mat".format(state))
    weak_mat_results.loc[state] = mat['rel_lower_MI_bounds'],  mat['rel_upper_MI_bounds'],  mat['rel_lower_MSE_bounds'],  mat['rel_upper_MSE_bounds']

### predict feature relevancies for MI-results using thresholds analogue to our method

In [12]:
def predict(rangevector):
        prediction = np.zeros(len(rangevector))
        # Threshold for relevancy
        upper_epsilon = 0.1
        lower_epsilon = 0.05

        # Weakly relevant ones have high upper bounds
        prediction[rangevector[:, 1] > upper_epsilon] = 1
        # Strongly relevant bigger than 0 + some epsilon
        prediction[rangevector[:, 0] > lower_epsilon] = 2

        allrel_prediction = prediction.copy()
        allrel_prediction[allrel_prediction == 2] = 1

        return allrel_prediction, prediction
    
def decision_function2(rangevector):
        upper_epsilon = 0.1
        lower_epsilon = 0.05
        dec1 = rangevector[:,0]-lower_epsilon
        dec2 = rangevector[:,1]-upper_epsilon

        return dec1
out = []
for state,datum in zip(test_range,str_rel_test_data):
    truth = datum[2]
    mirange = all_mat_results.loc[state].iloc[2:4].T.values
    pred,_ = predict(mirange)
    prec = precision_score(truth,pred)
    recall = recall_score(truth,pred)
    F1 = f1_score(truth,pred)
    accuracy = 0
    auc = roc_auc_score(truth, decision_function2(mirange))
    out.append([accuracy,prec,recall,F1,auc])
allRelData.loc["MutualInformation"] = np.array(out)
out = []
for state,datum in zip(test_range,str_rel_test_data):
    truth = datum[2]
    mirange = strong_mat_results.loc[state].iloc[2:4].T.values
    pred,_ = predict(mirange)
    prec = precision_score(truth,pred)
    recall = recall_score(truth,pred)
    F1 = f1_score(truth,pred)
    accuracy = 0
    auc = roc_auc_score(truth, decision_function2(mirange))
    out.append([accuracy,prec,recall,F1,auc])
strRelData.loc["MutualInformation"] = np.array(out)
out = []
for state,datum in zip(test_range,str_rel_test_data):
    truth = datum[2]
    mirange = weak_mat_results.loc[state].iloc[2:4].T.values
    pred,_ = predict(mirange)
    prec = precision_score(truth,pred)
    recall = recall_score(truth,pred)
    F1 = f1_score(truth,pred)
    accuracy = 0
    auc = roc_auc_score(truth, decision_function2(mirange))
    out.append([accuracy,prec,recall,F1,auc])
weakRelData.loc["MutualInformation"] = np.array(out)

NameError: name 'str_rel_test_data' is not defined

# Results

### Benchmark for dataset with only strongly relevant features

In [18]:
display(strRelData.mean().T)


,accuracy,precision,recall,F1 measure,auc
Ridge,0.942222,1.000000,0.833333,0.903636,0.511111
SVCL1,0.977778,0.569394,1.000000,0.724608,0.516667
ElasticN,0.911111,1.000000,0.883333,0.934545,0.508333
OurMethod,0.962222,1.000000,0.900000,0.943636,0.500000
BorutaModel,0.800000,0.985714,0.716667,0.808368,0.500000


### Benchmark for dataset with only weakly relevant features

In [19]:
display(weakRelData.mean().T)

,accuracy,precision,recall,F1 measure,auc
Ridge,0.955556,0.980000,0.800000,0.862727,0.369444
SVCL1,0.984444,0.569091,1.000000,0.724510,0.358333
ElasticN,0.975556,0.980000,0.833333,0.892727,0.358333
OurMethod,0.973333,1.000000,0.966667,0.980000,0.500000
BorutaModel,0.877778,1.000000,0.866667,0.920000,0.500000


### Benchmark for all relevant features

In [20]:
display(allRelData.mean().T)

,accuracy,precision,recall,F1 measure,auc
Ridge,0.948889,1.000000,0.700000,0.816061,0.594286
SVCL1,0.971111,0.657576,1.000000,0.792466,0.617143
ElasticN,0.946667,1.000000,0.757143,0.854429,0.597143
OurMethod,0.962222,1.000000,0.914286,0.951282,0.500000
BorutaModel,0.804444,0.980000,0.771429,0.858625,0.500000


## Output for paper in latex

In [21]:
datasets = [strRelData,weakRelData,allRelData]
for dataset in [strRelData,weakRelData,allRelData]:
    latex  = dataset.mean().T.drop(["accuracy","auc"],axis=1).round(2).to_latex()
    print(latex)


\begin{tabular}{lrrr}
\toprule
{} &  precision &  recall &  F1 measure \\
\midrule
Ridge       &       1.00 &    0.83 &        0.90 \\
SVCL1       &       0.57 &    1.00 &        0.72 \\
ElasticN    &       1.00 &    0.88 &        0.93 \\
OurMethod   &       1.00 &    0.90 &        0.94 \\
BorutaModel &       0.99 &    0.72 &        0.81 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrr}
\toprule
{} &  precision &  recall &  F1 measure \\
\midrule
Ridge       &       0.98 &    0.80 &        0.86 \\
SVCL1       &       0.57 &    1.00 &        0.72 \\
ElasticN    &       0.98 &    0.83 &        0.89 \\
OurMethod   &       1.00 &    0.97 &        0.98 \\
BorutaModel &       1.00 &    0.87 &        0.92 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrr}
\toprule
{} &  precision &  recall &  F1 measure \\
\midrule
Ridge       &       1.00 &    0.70 &        0.82 \\
SVCL1       &       0.66 &    1.00 &        0.79 \\
ElasticN    &       1.00 &    0.76 &        0.85 \\
OurMethod   &       1

In [22]:
d1 = strRelData.mean().T.drop(["accuracy","auc"],axis=1).rename(columns={"F1 measure":"F1"})
d2 = weakRelData.mean().T.drop(["accuracy","auc"],axis=1).rename(columns={"F1 measure":"F1"})
d3 = allRelData.mean().T.drop(["accuracy","auc"],axis=1).rename(columns={"F1 measure":"F1"})
new_frame = pd.concat([d1,d2,d3],axis=1,keys=['Dataset 1',"Dataset 2","Dataset 3"]).round(2)
print(new_frame.to_latex())

\begin{tabular}{lrrrrrrrrr}
\toprule
{} & Dataset 1 &        &       & Dataset 2 &        &       & Dataset 3 &        &       \\
{} & precision & recall &    F1 & precision & recall &    F1 & precision & recall &    F1 \\
\midrule
Ridge       &      1.00 &   0.83 &  0.90 &      0.98 &   0.80 &  0.86 &      1.00 &   0.70 &  0.82 \\
SVCL1       &      0.57 &   1.00 &  0.72 &      0.57 &   1.00 &  0.72 &      0.66 &   1.00 &  0.79 \\
ElasticN    &      1.00 &   0.88 &  0.93 &      0.98 &   0.83 &  0.89 &      1.00 &   0.76 &  0.85 \\
OurMethod   &      1.00 &   0.90 &  0.94 &      1.00 &   0.97 &  0.98 &      1.00 &   0.91 &  0.95 \\
BorutaModel &      0.99 &   0.72 &  0.81 &      1.00 &   0.87 &  0.92 &      0.98 &   0.77 &  0.86 \\
\bottomrule
\end{tabular}



In [23]:
new_frame


Dataset 1              Dataset 2              Dataset 3         \
            precision recall    F1 precision recall    F1 precision recall   
Ridge            1.00   0.83  0.90      0.98   0.80  0.86      1.00   0.70   
SVCL1            0.57   1.00  0.72      0.57   1.00  0.72      0.66   1.00   
ElasticN         1.00   0.88  0.93      0.98   0.83  0.89      1.00   0.76   
OurMethod        1.00   0.90  0.94      1.00   0.97  0.98      1.00   0.91   
BorutaModel      0.99   0.72  0.81      1.00   0.87  0.92      0.98   0.77   

                   
               F1  
Ridge        0.82  
SVCL1        0.79  
ElasticN     0.85  
OurMethod    0.95  
BorutaModel  0.86